In [1]:
import datetime as dt
import pandas as pd
import helper
import data

In [2]:
def get_christmas_date(year):
    # Add year to 12-25
    return dt.date(year, 12, 25)

def get_dates(beg_year = 2021):
    pass

In [3]:
data.Data.df = helper.get_initial_df()
df = data.Data.df
df.head(2)

In [20]:
df.sort_values(by='Date').reset_index(drop=True).head(16).Provider.value_counts()

William Halacoglu    3
Sabrina Taylor       2
John Jobes           2
Harold Woo           2
Norlan Maltez        1
Patricia Kahn        1
Sagar Patel          1
Adam Berkovits       1
Nelson Huang         1
Joseph Tran          1
Michael Tang         1
Name: Provider, dtype: int64

In [36]:
(dt.datetime.now().year) -1


2022